# Assign ISO to GTSM / CIAM points
- assign based on country Voronoi
- countries are defined by impact regions

INPUTS
- `sset.PATH_CIAM_SITES`
- `sset.PATH_GADM_ADM0_VORONOI`

OUTPUTS
- `sset.PATH_CIAM_SITES_WITHISO`

In [1]:
import geopandas as gpd

from sliiders import settings as sset
from sliiders import spatial

spatial.filter_spatial_warnings()

In [2]:
adm0 = gpd.read_file(sset.PATH_GADM, layer="level0", ignore_geometry=True)

In [4]:
adm0 = set(adm0['GID_0'].unique())

In [5]:
ciam_adm0 = set(gpd.read_parquet(sset.PATH_CIAM_SITES_WITHISO)['ISO'].unique())

In [6]:
adm0 - ciam_adm0

{'AFG',
 'AND',
 'ARM',
 'ATA',
 'AUT',
 'AZE',
 'BDI',
 'BFA',
 'BLR',
 'BOL',
 'BTN',
 'BWA',
 'CAF',
 'CHE',
 'CZE',
 'ETH',
 'HUN',
 'KAZ',
 'KGZ',
 'LAO',
 'LIE',
 'LSO',
 'LUX',
 'MAC',
 'MDA',
 'MKD',
 'MLI',
 'MNG',
 'MWI',
 'NER',
 'NPL',
 'PRY',
 'RWA',
 'SMR',
 'SRB',
 'SSD',
 'SVK',
 'SWZ',
 'TCD',
 'TJK',
 'TKM',
 'UGA',
 'UZB',
 'VAT',
 'XCA',
 'XKO',
 'XPI',
 'ZMB',
 'ZWE'}

In [ ]:
countries = gpd.read_parquet(sset.PATH_GADM_ADM0_VORONOI).set_crs(epsg=4326)

In [ ]:
stations = spatial.match_stations_to_countries(countries)

In [ ]:
sset.PATH_CIAM_SITES_WITHISO.parent.mkdir(exist_ok=True)

stations.to_parquet(sset.PATH_CIAM_SITES_WITHISO, index=False)